## INT420-491 : Categorical Data Encoding
45-minutes study

### Recap Imputation Missing Values

#### Define Functions

In [1]:
import pandas as pd 
from myfunc62130500048 import missing_value_summary, fillna_global_mean, fillna_group_mean, fillna_global_mode, fillna_group_mode, fillna_knn
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
#pd.set_option('mode.chained_assignment','warn')
pd.set_option('mode.chained_assignment',None)

### Read Data

In [2]:
from sklearn.preprocessing import LabelEncoder
df_org = pd.read_csv("train.csv")
df = df_org.copy()

In [3]:
missing_value_summary(df)

Your data contain 12 columns.
There are missing values in 2 columns.


,Total Missing Values,% of Missing Values
Outlet_Size,2410,28.28
Item_Weight,1463,17.17


In [4]:
df.head(3)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [5]:
df = fillna_global_mean(df, 'Item_Weight')

In [6]:
df[['Item_Weight']].head(10)

,Item_Weight
0,9.300000
1,5.920000
2,17.500000
3,19.200000
4,8.930000
5,10.395000
6,13.650000
7,12.857645
8,16.200000
9,19.200000


In [11]:
df = fillna_group_mean(df,'Item_Weight','Item_Type')

In [12]:
df[['Item_Weight']].head(10)

,Item_Weight
0,9.30000
1,5.92000
2,17.50000
3,19.20000
4,8.93000
5,10.39500
6,13.65000
7,12.98788
8,16.20000
9,19.20000


### Handle Missing Values 

In [13]:
print('Before: ') 
print(missing_value_summary(df))
df = fillna_global_mean(df, 'Item_Weight')
df = fillna_global_mode(df, 'Outlet_Size')
print('After: ') 
print(missing_value_summary(df))

Before: 
Your data contain 12 columns.
There are missing values in 1 columns.
             Total Missing Values  % of Missing Values
Outlet_Size                  2410                28.28
After: 
Your data contain 12 columns.
There are missing values in 0 columns.
Empty DataFrame
Columns: [Total Missing Values, % of Missing Values]
Index: []


## Encoding Categorical Data

### Handle Categorical Data by **Replacing values**
label as the given values

In [14]:
# 'Outlet_Size'
print(df['Outlet_Size'].unique())
# 'Small'==>2, 'Medium'=>5, 'High'==> 10

df['Outlet_Size'][df['Outlet_Size']=='Small'] = 2
df['Outlet_Size'][df['Outlet_Size']=='Medium'] = 5
df['Outlet_Size'][df['Outlet_Size']=='High'] = 10


['Medium' 'High' 'Small']


### Handle Categorical Data by **Label Encoding**
label as $0$ to $n-1$ where $n$ is the number of categories.

In [3]:
#'Item_Type'
df[['Item_Type']].head()

lb_maker = LabelEncoder()
df['Item_Type'] = lb_maker.fit_transform(df['Item_Type'])
df[['Item_Type']].head()
# print(lb_maker.classes_)


,Item_Type
0,4
1,14
2,10
3,6
4,9


In [4]:
df['Item_Type'] = lb_maker.inverse_transform(df['Item_Type'])
df[['Item_Type']].head()

,Item_Type
0,Dairy
1,Soft Drinks
2,Meat
3,Fruits and Vegetables
4,Household


### Handle Categorical Data by **One-Hot encoding**
generate $n$ columns with 0/1 where $n$ is the number of categories. 

In [7]:
df = pd.get_dummies(df, columns=['Item_Fat_Content','Outlet_Type'], 
                    prefix = ['Item_Fat_Content','Outlet_Type'])

In [8]:
df.head(20)

,Item_Identifier,Item_Weight,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Item_Outlet_Sales,Item_Fat_Content_LF,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Fat_Content_low fat,Item_Fat_Content_reg,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,FDA15,9.30,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,3735.1380,0,1,0,0,0,0,1,0,0
1,DRC01,5.92,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,443.4228,0,0,1,0,0,0,0,1,0
2,FDN15,17.50,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,2097.2700,0,1,0,0,0,0,1,0,0
3,FDX07,19.20,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,732.3800,0,0,1,0,0,1,0,0,0
4,NCD19,8.93,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,994.7052,0,1,0,0,0,0,1,0,0


### Handle Categorical Data by **Binary Encoding**
generate binary encoded vector, the number of columns = $\log_2 n$

In [7]:
df['Outlet_Location_Type'].unique()

array(['Tier 1', 'Tier 3', 'Tier 2'], dtype=object)

In [9]:
import category_encoders as ce
encoder = ce.BinaryEncoder(cols=['Outlet_Location_Type'])
df = encoder.fit_transform(df)

In [10]:
df.head()

,Item_Identifier,Item_Weight,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type_0,Outlet_Location_Type_1,Item_Outlet_Sales,Item_Fat_Content_LF,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Fat_Content_low fat,Item_Fat_Content_reg,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,FDA15,9.30,0.016047,Dairy,249.8092,OUT049,1999,Medium,0,1,3735.1380,0,1,0,0,0,0,1,0,0
1,DRC01,5.92,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,1,0,443.4228,0,0,1,0,0,0,0,1,0
2,FDN15,17.50,0.016760,Meat,141.6180,OUT049,1999,Medium,0,1,2097.2700,0,1,0,0,0,0,1,0,0
3,FDX07,19.20,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,1,0,732.3800,0,0,1,0,0,1,0,0,0
4,NCD19,8.93,0.000000,Household,53.8614,OUT013,1987,High,1,0,994.7052,0,1,0,0,0,0,1,0,0
